In [15]:
!pip install torchmetrics
!pip install tqdm
!pip install tensorflow


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python -m pip install --upgrade pip


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as  mape
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR, LinearSVR

from sklearn.linear_model import LinearRegression
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchmetrics import MeanSquaredLogError
from tqdm import tqdm
from torch.optim import AdamW

import tensorflow as tf
from tensorflow import keras as ks
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
import seaborn as sns


In [17]:
from preprocess import EstateData

dataset = EstateData("final.csv")

In [18]:
dataset.split_data(stratify=True)

In [19]:
dataset.preprocess(tukey=True)

Start Process!
Fill missing values: Done
Remove outlier: Done
Scale features: Done


In [20]:
train = dataset.train
test = dataset.test

In [21]:
print("train size {}".format(train.shape))
print("test size {}".format(test.shape))

train size (31531, 13)
test size (9278, 13)


In [22]:
train, dev = train_test_split(train, test_size=0.1)

In [23]:
train = pd.get_dummies(train)
dev = pd.get_dummies(dev)
test = pd.get_dummies(test)

In [24]:
features = train.columns.tolist()
features.remove("Price")
features.remove("PricePerM2")

In [25]:
model = ks.Sequential()
model.add(ks.layers.Dense(100, input_dim = 64, activation = 'relu', name = 'input'))
model.add(ks.layers.Dropout(0.25))
model.add(ks.layers.Dense(100, activation = 'relu', name = 'hidden1'))
model.add(ks.layers.Dropout(0.25))
model.add(ks.layers.Dense(100, activation = 'relu', name = 'hidden2'))
model.add(ks.layers.Dropout(0.2))
model.add(ks.layers.Dense(100, activation = 'relu', name = 'hidden3'))
model.add(ks.layers.Dropout(0.2))
model.add(ks.layers.Dense(1, name = 'output'))
opt = ks.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer = opt, 
              loss = tf.keras.losses.MeanSquaredLogarithmicError(), 
              metrics = ['mean_absolute_percentage_error'])
train_history = model.fit(train[features], train["PricePerM2"], batch_size = 2048, epochs = 200, validation_split = 0.1)
ann_train_res = model.evaluate(dev[features], dev["PricePerM2"])
ann_test_res = model.evaluate(test[features], test["PricePerM2"])
print("Train:  Loss = %f, Mean_absolute percentage_error = %f" %(ann_train_res[0], ann_train_res[1]/100))
print("Test:  Loss = %f, Mean_absolute_perntage_error = %f" %(ann_test_res[0], ann_test_res[1]/100))

Epoch 1/200
13/13 [==============================] - 3s 42ms/step - loss: 7.6893 - mean_absolute_percentage_error: 357.8672 - val_loss: 3.6961 - val_mean_absolute_percentage_error: 782.9787
Epoch 2/200
13/13 [==============================] - 0s 22ms/step - loss: 2.4520 - mean_absolute_percentage_error: 465.0164 - val_loss: 1.1848 - val_mean_absolute_percentage_error: 115.0715
Epoch 3/200
13/13 [==============================] - 0s 22ms/step - loss: 1.2535 - mean_absolute_percentage_error: 96.3591 - val_loss: 0.9153 - val_mean_absolute_percentage_error: 93.2101
Epoch 4/200
13/13 [==============================] - 0s 22ms/step - loss: 0.9014 - mean_absolute_percentage_error: 122.2116 - val_loss: 0.7386 - val_mean_absolute_percentage_error: 104.6066
Epoch 5/200
13/13 [==============================] - 0s 23ms/step - loss: 0.7419 - mean_absolute_percentage_error: 100.3557 - val_loss: 0.6257 - val_mean_absolute_percentage_error: 80.0723
Epoch 6/200
13/13 [==============================] - 

In [26]:
model.save('checkpoint/ANN.model')
model = ks.models.load_model('checkpoint/ANN.model')

INFO:tensorflow:Assets written to: checkpoint/ANN.model/assets


INFO:tensorflow:Assets written to: checkpoint/ANN.model/assets


In [27]:
pred = model.predict(test[features])
print(mape(test["PricePerM2"],pred ))


290/290 [==============================] - 0s 1ms/step
0.5624240997561355
